## Notes:

Limiting scope: 
Kern County 
2010 - present

# Library Imports

In [2]:
import numpy as np
import pandas as pd

# Data Import checks

## Drinking Water Data

In [14]:
raw_drinkchem = pd.read_excel('./assets/Drinkingwater/chemical.xlsx')

In [15]:
print(raw_drinkchem.shape)
raw_drinkchem.head()

(1048575, 12)


,PRIM_STA_C,SAMP_DATE,SAMP_TIME,LAB_NUM,ANADATE,INDATE,METHOD,INBY,SPECIAL,STORE_NUM,XMOD,FINDING
0,0103039-004,2017-11-22,848,4790,2017-11-22,2017-11-30,SM 212,#,S,00081,<,3.0
1,0103039-004,2017-11-22,848,4790,2017-11-22,2017-11-30,SM 215,#,S,00086,<,1.0
2,0103039-004,2017-11-22,848,4790,2017-11-29,2017-11-30,SM 251,#,S,00095,NaN,550.0
3,0103039-004,2017-11-22,848,4790,2017-11-29,2017-11-30,SM4500,#,S,00403,NaN,8.1
4,0103039-004,2017-11-22,848,4790,2017-11-22,2017-11-30,300.0,#,S,00618,NaN,5.7


In [6]:
raw_drinkchem.columns

Index(['PRIM_STA_C', 'SAMP_DATE', 'SAMP_TIME', 'LAB_NUM', 'ANADATE', 'INDATE',
       'METHOD', 'INBY', 'SPECIAL', 'STORE_NUM', 'XMOD', 'FINDING'],
      dtype='object')

In [17]:
raw_drinkchem['PRIM_STA_C'].unique()

array(['0103039-004', '0103040-007', '0103041-001', ..., '1510005-031',
       '1510005-032', '1510005-033'], dtype=object)

In [24]:
# According to the WQM data dictionary, Kern county code is 15. So all Primary station code / source number will start with 15
kern_sample = [sample for sample in raw_drinkchem['PRIM_STA_C'] if sample.startswith('15')]

In [26]:
len(kern_sample)

196389

In [41]:
raw_drinksys = pd.read_excel('./assets/Drinkingwater/watsys.xlsx')

In [42]:
print(raw_drinksys.shape)
raw_drinksys.head()

(15056, 11)


,SYSTEM_NO,SYSTEM_NAM,HQNAME,ADDRESS,CITY,STATE,ZIP,ZIP_EXT,POP_SERV,CONNECTION,AREA_SERVE
0,2800064,DAKOTA SHY WINERY,NaN,771 Sage Canyon Road,ST. HELENA,CA,94574,NaN,30.0,1.0,NaN
1,5403214,NaN,THREE RIVERS HISTORICAL MUSEUM,42268 SIERRA DR,THREE RIVERS,CA,93271,NaN,25.0,2.0,NaN
2,4000829,VINES R.V. RESORT,NaN,88 WELLSONA,PASO RO,CA,93446,NaN,400.0,1.0,NaN
3,0103039,MOHRLAND MUTUAL WATER SYSTEM,MOHRLAND MUTUAL,24927 Mohr Drive,HAYWARD,CA,94545,tt,116.0,99.0,NaN
4,0103040,NORRIS CANYON PROPERTY OWNERS ASSN.,NORRIS CANYON P,8653 Norris Canyon Road,CASTRO VALLEY,CA,94552,S,50.0,19.0,NaN


In [37]:
raw_drinksys['SYSTEM_NO'].unique()

array(['2800064', '5403214', '4000829', ..., '3107347', '1000644',
       '1504009'], dtype=object)

In [43]:
kern_sys = [facility for facility in raw_drinksys['SYSTEM_NO'] if facility.startswith('15')]

In [46]:
len(kern_sys)

688

In [92]:
raw_drinksys['AREA_SERVE'].value_counts()

WASTEWATER GENERATOR                       198
Wastewater Generator                        22
USFS-White Mountain Ranger District         10
NC PARK RES                                  8
Inyo County Parks & Rec                      7
T9                                           5
NC CMPG                                      5
USFS-Mount Whitney Ranger District           5
BELLFLOWER                                   4
State of CA-Dept. of Parks & Recreation      4
CAMARILLO                                    3
Western MWD                                  3
R1                                           3
California Water Service Company             3
EL RIO                                       3
MONTEBELLO                                   3
Ken Wilder                                   3
FRESNO                                       3
T4                                           3
City of Hollister                            2
Mariah s Meadow                              2
LOS LAGOS    

Right off I'm sure I'll need SystemNo, which is county number + sys type + sequence number, so again only need those that start with 15. If I have that I won't need system name, HQ name, address, city, state, zip, or zip ext. I could extract a really rough location from the address (lat, long). Population served is important and probably the only other important info from this table

In [51]:
raw_drinksite = pd.read_excel('./assets/Drinkingwater/siteloc.xlsx')

In [52]:
print(raw_drinksite.shape)
raw_drinksite.head()

(58328, 11)


,PRI_STA_C,FRDS_NO,COUNTY,DISTRICT,USER_ID,SYSTEM_NO,WATER_TYPE,SOURCE_NAM,STATION_TY,STATUS,COMMENT_1
0,4301014-001,4301014001,43,17,SAN,4301014,G,WELL 01 - TREATED,NaN,AT,NaN
1,4301014-002,4301014002,43,17,SAN,4301014,G,WELL 01- CL2 TREATMENT,NaN,DT,NaN
2,2300906-002,2300906002,23,3,RXR,2300906,G,DBP SAMPLE LOCATION,NaN,DT,NaN
3,2310006-030,2310006030,23,3,RXR,2310006,G,DBP 781 TOKAY DRIVE,NaN,DT,NaN
4,2300955-001,2300955001,23,3,RXR,2300955,G,WELL 01,NaN,AR,NaN


In [55]:
kern_sites = raw_drinksite[raw_drinksite['COUNTY'] == 15]

In [56]:
len(kern_sites)

3241

In [58]:
kern_sites.head(10)

,PRI_STA_C,FRDS_NO,COUNTY,DISTRICT,USER_ID,SYSTEM_NO,WATER_TYPE,SOURCE_NAM,STATION_TY,STATUS,COMMENT_1
31,1503226-004,1503226004,15,19,TEH,1503226,G,WELL 01- UMTALI WELL CL2 TREATMENT XCLD,NaN,AT,NaN
32,1503226-005,1503226005,15,19,TEH,1503226,G,WELL 02- MONTCLAIRE WELL CL2 TREATMENT,NaN,AT,NaN
33,1503226-901,1503226901,15,19,TEH,1503226,G,ST2S1- COUNTRY CANYON & UMTALI SAMPLE ST,NaN,DT,NaN
73,1503576-005,1503576005,15,19,TEH,1503576,G,WELL 02,NaN,AR,NaN
93,1504009-001,1504009001,15,12,CYA,1504009,G,WELL 001,NaN,AR,NaN
94,1510055-038,1510055038,15,12,CYA,1510055,G,WELL 201-01 - GAC_EFF-TCP & H2S,NaN,DT,NaN
183,1503684-001,1503684001,15,19,CYA,1503684,G,WELL 01,NaN,AR,NaN
193,1510028-014,1510028014,15,19,TEH,1510025,G,WELL 09,NaN,AR,NaN
194,1510028-015,1510028015,15,19,TEH,1510028,G,WELL 10,NaN,AR,NaN
196,1510003-267,1510003267,15,12,CYA,1510003,G,WELL 202-01 - AS-TRT,NaN,AT,NaN


In [63]:
kern_sites['COMMENT_1'].unique()

array([nan], dtype=object)

Care about:
Water Type = G, will need to remove sites where status are things like "abandoned" or "destroyed" etc. 
Status = AR (Active Raw), AU (Active Untreated)

Drop: FRDS_NO, County, District, User, Comments.

Not sure how to handle Station Type just yet. There's a lot to parse

In [59]:
kern_sites['STATUS'].value_counts()

AR    727
AT    722
DT    374
AU    279
IU    277
IR    245
IT    151
DS    147
AB     81
PN     43
SR     34
PT     34
CT     29
SU     28
ST     17
MW     13
CR     13
AG     10
WW      9
PR      5
DR      2
CU      1
Name: STATUS, dtype: int64

In [77]:
kern_sites.columns

Index(['PRI_STA_C', 'FRDS_NO', 'COUNTY', 'DISTRICT', 'USER_ID', 'SYSTEM_NO',
       'WATER_TYPE', 'SOURCE_NAM', 'STATION_TY', 'STATUS', 'COMMENT_1'],
      dtype='object')

In [78]:
kern_sites.drop(['FRDS_NO', 'COUNTY', 'DISTRICT', 'USER_ID', 'COMMENT_1'], axis=1, inplace=True)

C:\Users\Nicole\Anaconda3\envs\dsi\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).drop(labels=labels, axis=axis,


In [85]:
kern_sites = kern_sites[kern_sites['WATER_TYPE'] == 'G']

In [88]:
kern_sites = kern_sites[(kern_sites['STATUS'] == 'AR') | (kern_sites['STATUS'] == 'AU')]

In [89]:
kern_sites.shape

(991, 6)

In [90]:
kern_sites.head()

,PRI_STA_C,SYSTEM_NO,WATER_TYPE,SOURCE_NAM,STATION_TY,STATUS
73,1503576-005,1503576,G,WELL 02,NaN,AR
93,1504009-001,1504009,G,WELL 001,NaN,AR
183,1503684-001,1503684,G,WELL 01,NaN,AR
193,1510028-014,1510025,G,WELL 09,NaN,AR
194,1510028-015,1510028,G,WELL 10,NaN,AR


In [91]:
kern_sites.drop('WATER_TYPE', axis=1, inplace=True)

## Groundwater Data

In [7]:
raw_gw = pd.read_csv('./assets/Groundwater/KernEDF.txt', sep='\t')

In [8]:
raw_gw.head()

,COUNTY,GLOBAL_ID,FIELD_PT_NAME,LOGDATE,LOGTIME,LOGCODE,SAMPID,MATRIX,LABWO,LABCODE,...,ANADATE,BASIS,PARLABEL,PARVAL,PARVQ,LABDL,REPDL,UNITS,DILFAC,LNOTE
0,Kern,GAGW10000018,NCL-2-02,2014-11-21,930.0,BTSC,NCL-2-02,W,94042-1,TAIC,...,2014-11-25,N,BZ,0.000,ND,0.250,2.0,UG/L,1.0,NaN
1,Kern,GAGW10000018,NCL-2-02,2014-11-21,930.0,BTSC,NCL-2-02,W,94042-1,TAIC,...,2014-11-25,N,XYLO,0.000,ND,0.250,2.0,UG/L,1.0,NaN
2,Kern,GAGW10000018,NCL-2-02,2014-11-21,930.0,BTSC,NCL-2-02,W,94042-1,TAIC,...,2014-12-23,N,RA226228,0.691,=,0.377,5.0,PCI/L,1.0,NaN
3,Kern,GAGW10000018,NCL-28E4,2014-09-12,1000.0,BTSC,NCL-28E4,W,88029-1,TAIC,...,2014-09-13,N,HCO3,0.000,ND,4.800,4.8,MG/L,1.0,NaN
4,Kern,GAGW10000018,NCL-2-02,2016-05-10,908.0,BTST,Monitoring Well #2-02,W,NaN,BCLB,...,2016-05-17,N,CR,1.600,=,1.100,10.0,UG/L,1.0,"J,DX"


In [18]:
raw_gw.columns

Index(['COUNTY', 'GLOBAL_ID', 'FIELD_PT_NAME', 'LOGDATE', 'LOGTIME', 'LOGCODE',
       'SAMPID', 'MATRIX', 'LABWO ', 'LABCODE', 'LABSAMPID', 'ANMCODE',
       'LABLOTCTL', 'ANADATE', 'BASIS', 'PARLABEL', 'PARVAL', 'PARVQ', 'LABDL',
       'REPDL', 'UNITS', 'DILFAC', 'LNOTE'],
      dtype='object')

In [19]:
raw_gw['LOGDATE'].unique()

array(['2014-11-21', '2014-09-12', '2016-05-10', ..., '2006-06-30',
       '2013-06-12', nan], dtype=object)

In [36]:
raw_gw['FIELD_PT_NAME'].unique()

array(['NCL-2-02', 'NCL-28E4', 'QCTB', ..., 'A2-SW16', 'OW-21', nan],
      dtype=object)

In [27]:
raw_gw_loc = pd.read_excel('./assets/Groundwater/gama_location_kern_county.xlsx')

In [28]:
raw_gw_loc.shape

(1050, 5)

In [29]:
raw_gw_loc.head()

,DATASET,WELL ID,WELL NAME,LATITUDE,LONGITUDE
0,USGSNEW,USGS-345733118085201,USGS-345733118085201,34.959140,-118.148686
1,USGSNEW,USGS-351824117570101,USGS-351824117570101,35.306628,-117.951180
2,DHS,W0601502244,1502244-001,35.602611,-119.507496
3,DHS,W0601503431,1503431-003,35.715251,-119.427021
4,DHS,W0601510302,1510302-001,35.332059,-119.362728


In [35]:
raw_gw_loc['WELL NAME'].unique()

array(['USGS-345733118085201', 'USGS-351824117570101', '1502244-001', ...,
       '1510703-003', '1510703-001', '1502608-001'], dtype=object)

# Creating a SQL database

This is to store and more easily query all the different databases instead of doing it all in Pandas

In [67]:
from sqlalchemy import create_engine